In [1]:
from pyspark import SparkContext
sc = SparkContext('local')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/20 05:55:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/12/20 05:55:10 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/12/20 05:55:10 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [2]:
empList=['1,kim,000-000-000','2,lee,111-111-111',
       '3,park,222-222-222','4,song,333-333-333',
       '5,han,555-555-555','6,yoon,666-666-666']

deptList = ['1,sale','2,developer','3,manager',
           '4,sale1','5,developer1','6,manager1']

In [3]:
emprdd = sc.parallelize(empList)
deptrdd = sc.parallelize(deptList)

In [4]:
emprdd1 = emprdd.map(lambda v:v.split(','))
emprdd1.collect()

[['1', 'kim', '000-000-000'],
 ['2', 'lee', '111-111-111'],
 ['3', 'park', '222-222-222'],
 ['4', 'song', '333-333-333'],
 ['5', 'han', '555-555-555'],
 ['6', 'yoon', '666-666-666']]

In [5]:
emprdd2=emprdd1.map(lambda v:(v[0],v[1]+' '+v[2]))
emprdd2.take(3)

[('1', 'kim 000-000-000'), ('2', 'lee 111-111-111'), ('3', 'park 222-222-222')]

In [6]:
deptrdd1=deptrdd.map(lambda v : v.split(',')).map(lambda v : (v[0],v[1]))
deptrdd1.take(3)

[('1', 'sale'), ('2', 'developer'), ('3', 'manager')]

In [7]:
join_data=emprdd2.join(deptrdd1)
join_data.take(3)

[('1', ('kim 000-000-000', 'sale')),
 ('4', ('song 333-333-333', 'sale1')),
 ('2', ('lee 111-111-111', 'developer'))]

In [8]:
sortrdd = join_data.sortByKey()
sortrdd.take(3)

[('1', ('kim 000-000-000', 'sale')),
 ('2', ('lee 111-111-111', 'developer')),
 ('3', ('park 222-222-222', 'manager'))]

In [10]:
weightrdd = sc.textFile('./main/data/weight.csv')
weightrdd.collect()

['year,height,weight,grade,gender,gradecode,gendercode',
 '2017,152.5,47.9,elementary,man,2,1',
 '2017,153.2,46.6,elementary,woman,2,0',
 '2017,170.6,63.8,middle,man,0,1',
 '2017,160.4,54.2,middle,woman,0,0',
 '2017,173.9,72.3,high,man,1,1',
 '2017,160.9,57.7,high,woman,1,0']

In [ ]:
weightrdd.map(lambda x : x.split(',')).take(3)

[['year', 'height', 'weight', 'grade', 'gender', 'gradecode', 'gendercode'],
 ['2017', '152.5', '47.9', 'elementary', 'man', '2', '1'],
 ['2017', '153.2', '46.6', 'elementary', 'woman', '2', '0']]

In [ ]:
weightrdd.map(lambda x:x.split(',')).zipWithIndex().collect()

[(['year', 'height', 'weight', 'grade', 'gender', 'gradecode', 'gendercode'],
  0),
 (['2017', '152.5', '47.9', 'elementary', 'man', '2', '1'], 1),
 (['2017', '153.2', '46.6', 'elementary', 'woman', '2', '0'], 2),
 (['2017', '170.6', '63.8', 'middle', 'man', '0', '1'], 3),
 (['2017', '160.4', '54.2', 'middle', 'woman', '0', '0'], 4),
 (['2017', '173.9', '72.3', 'high', 'man', '1', '1'], 5),
 (['2017', '160.9', '57.7', 'high', 'woman', '1', '0'], 6)]

In [ ]:
data=weightrdd.map(lambda x : x.split(',')).filter(lambda x : x[0]!='year')

In [ ]:
data.map(lambda x : float(x[1])).sum()

971.4999999999999

In [ ]:
data.map(lambda x : float(x[1])).mean()

161.91666666666666

In [ ]:
data.map(lambda x : x[3]).countByValue()

defaultdict(int, {'elementary': 2, 'middle': 2, 'high': 2})

In [ ]:
data.sortBy(lambda x:x[2],ascending=False).collect()

[['2017', '173.9', '72.3', 'high', 'man', '1', '1'],
 ['2017', '170.6', '63.8', 'middle', 'man', '0', '1'],
 ['2017', '160.9', '57.7', 'high', 'woman', '1', '0'],
 ['2017', '160.4', '54.2', 'middle', 'woman', '0', '0'],
 ['2017', '152.5', '47.9', 'elementary', 'man', '2', '1'],
 ['2017', '153.2', '46.6', 'elementary', 'woman', '2', '0']]

In [ ]:
# 전체 키 합과 평균
# (ele 2 mid 2 high 2)
# 몸무게 높은순